In [2]:
%load_ext autoreload
%autoreload 2
import sys
import tensorflow as tf
import h5py, os, yaml
import umap.umap_ as umap
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import re
from modelzoo import GELU
import seaborn as sns
from scipy import stats
from test_to_bw_fast import read_model, get_config
import pandas as pd
import matplotlib.patches as patches
from tensorflow import keras
import plotly.express as px
import tfr_evaluate, util
import subprocess
import explain
from tqdm import tqdm
import glob
import embed
from scipy.stats import pearsonr
import metrics
import test_to_bw_fast
import embed

In [3]:
testset, targets = tfr_evaluate.collect_whole_testset(data_dir='/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_dataset/i_2048_w_1/', coords=False)


In [4]:
colab_models = tfr_evaluate.collect_run_dirs('COLAB_MODEL_SELECTION', wandb_dir='/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/wandb/wandb_colab/*')


wandb: Currently logged in as: toneyan (use `wandb login --relogin` to force relogin)


In [100]:
all_dfs = []
for run_dir in colab_models:
    model, bin_size = test_to_bw_fast.read_model(run_dir)
    complete_performance, scaling_factors = tfr_evaluate.evaluate_run_whole(model, bin_size, testset, targets)

    eval_type = 'whole'
    pred_type = 'raw'

    raw_whole_perf = complete_performance[(complete_performance['pred type']==pred_type)]

    raw_whole_perf['run_dir'] = run_dir
    raw_whole_perf['bin_size'] = bin_size
    all_dfs.append(raw_whole_perf)
complete_performance = pd.concat(all_dfs)
complete_performance.to_csv('/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/whole_eval.csv')

/home/shush/tf_2/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/shush/tf_2/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [3]:
complete_performance = pd.read_csv('/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/whole_eval.csv', index_col='Unnamed: 0')

summary_df = complete_performance.groupby(['run_dir', 'bin_size']).mean().reset_index().sort_values('pr_corr')[::-1]

summary_df['model'] = [tfr_evaluate.get_config(s)['model_fn']['value'] for s in summary_df['run_dir'].values]

In [4]:
summary_df

,run_dir,bin_size,mse,js_per_seq,js_conc,poiss,pr_corr,sp_corr,model
5,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32,20.033419,0.466013,0.403525,-1.147655,0.752273,0.358527,residual_profile_task_conv_32
4,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32,21.289374,0.463262,0.409500,-1.116898,0.750426,0.343279,residual_profile_all_dense_32
0,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,1,20.884892,0.479705,0.417046,-1.147827,0.745505,0.339317,residual_profile_task_base
3,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32,20.273808,0.480199,0.409590,-1.134167,0.736514,0.354762,bpnet
1,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,1,21.575338,0.503385,0.421875,-1.136762,0.726155,0.337294,bpnet
6,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32,27.755841,0.475173,0.474442,-0.743382,0.655922,0.276341,basenjimod
2,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,1,29.732543,0.511358,0.489145,-0.727290,0.643289,0.263557,basenji_w1_b64


In [4]:
best_model = summary_df.iloc[0,0]

In [7]:
complete_performance[complete_performance['run_dir'] == best_model].sort_values('pr_corr')[::-1]

,mse,js_per_seq,js_conc,poiss,pr_corr,sp_corr,targets,pred type,eval type,run_dir,bin_size
8,20.082785,0.512522,0.371540,-2.138304,0.849573,0.403489,CAGMYC_DD5_Virgin_Input_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
1,20.084110,0.512580,0.371577,-2.138130,0.849553,0.403634,CAGMYC_DD5_Virgin_Input_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
10,5.017448,0.368285,0.338329,-0.609346,0.839512,0.398253,CAGMYC_DD2_Virgin_Input_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
0,8.193499,0.454899,0.411459,-0.783151,0.813228,0.301316,CAGMYC_DD5_Parous_Input_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
9,26.433764,0.488462,0.386740,-1.851025,0.792936,0.382552,CAGMYC_DD2_Virgin_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
6,33.906020,0.496436,0.396267,-2.129042,0.782732,0.400961,CAGMYC_DD5_Parous_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
13,14.790830,0.478717,0.400335,-1.303158,0.779383,0.398062,BalbC_Parous_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
11,51.326336,0.522609,0.389863,-2.630073,0.758446,0.382937,CAGMYC_DD5_Virgin_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
5,51.344320,0.521415,0.389670,-2.630053,0.758406,0.383115,CAGMYC_DD5_Virgin_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32
14,21.436514,0.474887,0.401585,-1.497222,0.754632,0.363222,BalbC_During_2ndPreg_27ac,raw,whole,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...,32


# write to bw for visualization

In [8]:
model, bin_size = tfr_evaluate.read_model(best_model)

In [9]:
testset, targets = tfr_evaluate.collect_whole_testset(data_dir='/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_dataset/i_2048_w_1/', coords=True)



In [10]:
C, X, Y = util.convert_tfr_to_np(testset, 3)

In [11]:
preds = embed.predict_np(X, model, batch_size=32, reshape_to_2D=False)

In [12]:
targets

array(['CAGMYC_DD5_Parous_Input_27ac', 'CAGMYC_DD5_Virgin_Input_27ac',
       'CAGMYC_DD5_Parous_Rep1_ATAC', 'CAGMYC_DD5_Virgin_Rep1_ATAC',
       'CAGMYC_DD5_Parous_Rep2_ATAC', 'CAGMYC_DD5_Virgin_27ac',
       'CAGMYC_DD5_Parous_27ac', 'CAGMYC_DD5_Virgin_Rep2_ATAC',
       'CAGMYC_DD5_Virgin_Input_27ac', 'CAGMYC_DD2_Virgin_27ac',
       'CAGMYC_DD2_Virgin_Input_27ac', 'CAGMYC_DD5_Virgin_27ac',
       'BalbC_Virgin_27ac', 'BalbC_Parous_27ac',
       'BalbC_During_2ndPreg_27ac', 'BalbC_During_1stPreg_27ac'],
      dtype=object)

In [21]:
targets_i = np.argwhere(targets=='CAGMYC_DD2_Virgin_Input_27ac')[0][0]
out_dir = util.make_dir('/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/model_bws')
pred_bw_filename = os.path.join(out_dir, 'pred_{}.bw'.format(targets[targets_i]))
truth_bw_filename = os.path.join(out_dir, 'truth_{}.bw'.format(targets[targets_i]))

In [22]:
# open 2 bws
chrom_size_path='/home/shush/genomes/mm10.chrom.sizes'
pred_bw = test_to_bw_fast.open_bw(pred_bw_filename, chrom_size_path)
truth_bw = test_to_bw_fast.open_bw(truth_bw_filename, chrom_size_path)

In [23]:
clean_C = [str(c).strip('b\'').strip('\'') for c in C]

In [24]:
clean_C[0].split('_')

['chr8', '0', '2048']

In [25]:

for i in range(preds.shape[0]): # per data point 
    chrom, start, end = clean_C[i].split('_') # get chr, start, end
    start = int(start) # to feed into bw making function
    # for cell_line in [8]: # per cell line
    for t in [targets_i]: # per cell line
        # write to ground truth file
        truth_bw.addEntries(chrom, start,
            values=np.array(np.squeeze(Y[i,:,t]), dtype='float64'),
            span=1, step=1)
        # write to prediction bw file
        pred_bw.addEntries(chrom, start,
            values=np.array(np.squeeze(preds[i,:,t]), dtype='float64'),
            span=bin_size, step=bin_size)


In [26]:
truth_bw.close()
pred_bw.close()

# make saliency maps

In [48]:
saliency_cell_line_dir = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps'
util.make_dir(saliency_cell_line_dir)

'/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps'

In [29]:
[t for t in targets if 'DD5' in t and 'ATAC' not in t]

['CAGMYC_DD5_Parous_Input_27ac',
 'CAGMYC_DD5_Virgin_Input_27ac',
 'CAGMYC_DD5_Virgin_27ac',
 'CAGMYC_DD5_Parous_27ac',
 'CAGMYC_DD5_Virgin_Input_27ac',
 'CAGMYC_DD5_Virgin_27ac']

In [83]:
for cell_line in range(len(targets)):
    sets_of_ids = {'top_{}_preds'.format(targets[cell_line]): preds[:,:,cell_line].mean(axis=1).argsort()[::-1][:50]}
    for label,selected_sample_ids in sets_of_ids.items():
        output_path = os.path.join(saliency_cell_line_dir, label+'.svg')
        if os.path.isfile(output_path):
            print('Skipping '+output_path)
        else:
            print(output_path)
            explainer = explain.Explainer(model, class_index=cell_line)
            X_sample = X[selected_sample_ids]
            saliency_scores = explainer.saliency_maps(X_sample)
            explain.plot_saliency_logos_oneplot(saliency_scores, X_sample, window=256, 
                                            titles = selected_sample_ids,
                                            filename=output_path)

Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Parous_Input_27ac_preds.svg
Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Virgin_Input_27ac_preds.svg
Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Parous_Rep1_ATAC_preds.svg
Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Virgin_Rep1_ATAC_preds.svg
Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Parous_Rep2_ATAC_preds.svg
Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Virgin_27ac_preds.svg
Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Parous_27ac_preds.svg
Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Virgin_Rep2_ATAC_preds.svg
Skipping /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/saliency_maps/top_CAGMYC_DD5_Virgin_Input_27ac_

In [77]:
top50_preds = np.repeat(preds[sets_of_ids['top_CAGMYC_DD5_Parous_Input_27ac_preds']][:,:,0:1], bin_size, axis=1)

metrics.get_correlation_per_seq(top50_preds, Y[sets_of_ids['top_CAGMYC_DD5_Parous_Input_27ac_preds'],:,0:1])